## Part 2 

### In this part you will simulate a sample of T = 100 observations from a stationary AR(1) process with $\alpha=0.8$, $\sigma^2 = 1$

In [1]:
import numpy as np

Generate a $T$ vector of innovations using `gen = np.random.default_rng(100)`

In [2]:
T=100
gen = np.random.default_rng(100)
eps = gen.standard_normal(T)

Using $\varepsilon_0$, generate $z_0$ from the stationary distribution of $\mathbf{z}$. Then, using a `for` loop, generate the remaining 99 realizations of $\mathbf{z}$

In [3]:
alpha=0.8
Z0=eps[0]/np.sqrt(1-alpha**2)

realiz_Z = np.array([Z0])

for c in eps[1:T+1]:
    real = alpha * realiz_Z[-1]  + c
    realiz_Z=np.append(realiz_Z, real)
realiz_Z

array([-1.92924941e+00, -1.25364373e+00, -2.22060913e-01,  3.66324915e-01,
       -6.68322709e-01,  5.36350498e-01,  1.13053606e+00,  1.60940230e+00,
        2.03258444e+00,  2.73041479e+00,  4.42730423e+00,  2.93035026e+00,
        2.39149139e+00,  3.66742780e+00,  1.59596237e+00,  1.60234437e+00,
        5.92757776e-01,  4.54384411e-01,  8.38260775e-01, -1.26049280e+00,
       -2.00087252e+00, -3.00616909e+00, -2.63603078e+00, -2.79767171e+00,
       -7.23031590e-01, -1.18159682e+00,  7.68407033e-01,  2.08476435e-01,
        4.38190669e-01,  3.90392735e-01,  3.23832507e-01, -8.68112003e-01,
       -3.59776627e-01,  9.60702469e-02,  3.14691711e-01,  8.73165044e-01,
       -1.20713512e-01, -3.94150015e-01, -9.76882717e-01, -2.48567650e+00,
       -1.62100488e+00, -1.93229270e+00, -1.62355466e+00,  9.50924107e-01,
        9.91049815e-01,  9.00255752e-01,  1.79415038e+00,  2.68167422e+00,
        3.95823207e+00,  2.64510169e+00,  3.91201629e+00,  2.99825523e+00,
        1.24037739e+00,  


It would be nice if we could more easily build matrices with simple structure like that of $\boldsymbol B$ without having the specify each element individually.

$$
\underset{\boldsymbol B}{\underbrace{\left[
\begin{array}{cccccccc}
\sqrt{1-\alpha^2} & 0 & 0 & 0 & \cdots & 0 & 0 & 0\\
-\alpha & 1 & 0 & 0 & \cdots & 0 & 0 & 0\\
0 & -\alpha  & 1 & 0 & \cdots & 0 & 0 & 0\\
\vdots & \vdots & \vdots & \vdots & \cdots & \vdots & \vdots & \vdots\\
0 & 0  & 0 & 0 & \cdots & 0 & -\alpha & 1
\end{array}\right]}}
\underset{\boldsymbol z}{\underbrace{
\left[
\begin{array}{c}
z_{0}\\
z_{1}\\
z_{2}\\
\vdots\\
z_{T}
\end{array}
\right]
}}
=
\underset{\mathbf{\varepsilon}}{\underbrace{\left[
\begin{array}{c}
\varepsilon_{0}\\
\varepsilon_{1}\\
\varepsilon_{2}\\
\vdots\\
\varepsilon_{T}
\end{array}
\right]
}}
$$

Fortunately, we can. Ignoring the the (0,0) element for a moment, the rest of $\boldsymbol B$ can be created using `np.eye()`, `np.diag()` and `np.tile()` functions. After that, the (0,0) element can be added manually. Check these functions' [documentation](https://numpy.org/doc/stable/index.html) and create the $T \times T$ matrix $\boldsymbol B$.

In [6]:
B = np.eye(T) + np.diag(np.tile(-alpha, T-1), k=-1)
B[0,0] = np.sqrt(1-alpha**2)
B

array([[ 0.6,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       [-0.8,  1. ,  0. , ...,  0. ,  0. ,  0. ],
       [ 0. , -0.8,  1. , ...,  0. ,  0. ,  0. ],
       ...,
       [ 0. ,  0. ,  0. , ...,  1. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , ..., -0.8,  1. ,  0. ],
       [ 0. ,  0. ,  0. , ...,  0. , -0.8,  1. ]])

```{note}
There are other ways to quickly build such matrices. If you are interested, check the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.toeplitz.html) of the `toeplitz()` function from the `scipy.linalg` package
```

Verify that the equality $\mathbf{z} = \boldsymbol B^{-1} \boldsymbol\varepsilon$ holds

</div> <div class="alert alert-block alert-danger">

Here is how to use the `toeplitz()` function (could be useful if there were more than 2 non-zero diagonals in B)

```
from scipy.linalg import toeplitz
c = np.zeros(T)
c[0] = 1
c[1] = -alpha
r = np.zeros(T)
BB = toeplitz(c, r)
BB[0, 0] = np.sqrt(1 - alpha**2)

np.testing.assert_array_almost_equal(B, BB)
```

In [8]:
np.testing.assert_array_almost_equal(realiz_Z, np.linalg.inv(B) @ eps)

Using $\boldsymbol B$, compute the first T theoretical autocovariances of $z_t$

In [9]:
ar1_acov1 = np.linalg.inv(B) @ np.linalg.inv(B).transpose()[:,0]

assert len(ar1_acov1) == T



Using `statsmodels`, define an AR(1) object with the same parameters as given above, and compute the T theoretical autocovariances

In [10]:
from statsmodels.tsa.arima_process import ArmaProcess
import statsmodels.api as sm
import statsmodels.tsa.api as smt

ar = np.r_[1, -alpha] 
ma = np.r_[1]
ar1_process = ArmaProcess(ar, ma)
ar1_acov2 = ar1_process.acovf(T)



Confirm that `ar1_acov1` and `ar1_acov2` are the same (or numerically very close)

In [12]:
np.testing.assert_almost_equal(ar1_acov1, ar1_acov2)